# TRIVIATHON

## Team Member:
### Adil Hamid Malla,Sumeet Singh Arora, Rahul Bhagat



## Introduction and Problem Statement
Searching is very important part of present internet era. Most of the search engines try to give you
exact and only information you ask through the query. Providing additional interesting facts
along with required information makes user search more exploratory and help in increasing the
user engagement. The latest search engines have started giving additional information to the user
like www.yippy.com and others.

Today trivia games are quite popular and played throughout world from pubs to mobile
applications. It has been found that more than 50 percent of web search queries are entity based
[1,2]. Hence amplifying the search result with such facts facts would help in increasing the user
engagement and improve dwell time. Business case studies have shown that trivia helps improve
revenue and user engagement.


## Prior Work

Sergey Chernov et al. [3] suggested that semantic information can be extracted from Wikipedia by
analyzing the links between categories.

Prakash et al. [4] proposed the Wikipedia Trivia Miner(WTM) which is a supervised learning based
algorithm. WTM learns linguistic and entity-based features from a labelled dataset derived from
the IMDB trivia section.

Merzbacher [5] present a (nearly) domain-independent approach to mining trivia questions from a
database. Generated questions are ranked and are more “interesting” if they have a modest
number of solutions and may reasonably be solved (but are not too easy).

The Google search engine has also came up with the trivia facts if you run the query fun facts or trivia.

<img src="images/google.jpg">



## Basic Plan & Methodology

As we know that the trivia associated with an entity is always centered around the rare and typical knowledge. So we can say that the trivia facts have a surprise element associated with it, which represents the trivia-worthiness of the fact. Since we are dealing with the wikipedia data, we know that the facts are mostly the categories of the article. Now to calculate the trivia-worthiness of the category, we can simply choose the category which has very less number of articles- meaning that the category is rare and unique trait. But only taking the number into consideration is not good idea. So we have following two metric to calculate the trivia-worthiness of the categories.

As Tsurel et. al [6] described in their work, for calculating the trivia facts, we need to calulate the surprise factor related to each fact. To do so, we need a metric which measures the surprise factor of the article. As we know that the category is the collection of articles, like 1992 birth, which enlists all the people born in 1992, so, to get the surprise element, we will calculate the similarity of the articles in the category. We also measure the similarity between the articles and the categories. Finally, the surprise factor will be represented by the inverse of average similarity, thus giving as a category which is highly different from other categories.

The second metric is the calculation of the cohesiveness of the category, which is defined as the average similarity between the articles of the category. Cohesiveness describes the elusiveness of the facts. Now, we need a score which combines the two metric values for a category. Thus, the score metric is the multiplication of the cohesiveness of the category multpilied with that of surprise factor of the category.




## Data Scrapping Using Pywikibot and Wikipedia Libraries

We have used the pywikibot and wikipedia libraries for scrapping the data from the wikipedia. 

Once we get the input entity from the user for which we have to calculate the Trivia facts. For the first case we will check whether the entity exists in the wikipedia or not. Sometimes it might happen that the user typed wrong spellings or half name, so we use the wikipedia search api for getting the best and correct entity name that is an article in the wikipedia here.

Once we got the article entity here, we will proceed with the fetching the article text using the pywikibot api. We also get the categories that entity belong to. 

The next step involes the preprocessing of the retrieved data from the wikipedia.
First of all we need to remove the html tags and get the plain text from it. We have used Wiki2Plain[7] code for removing the html tags, images, punctuations and also unwiki the text, as it contains the text in some format like sections and subsection. 
The second step of preprocessing is lowercase all the text so that we have uniform cases. The third step is to remove the stop words. we have used the nltk library to remove the stop words. And the final preprocessing step is to do stemming using Porter Stemmer of the words and get the tokens for each article.

To enhance the computation of our algorithm we have used the cache at different levels. We have cached the stemming results. Moreover to reduce the api calls to fetch the data from wikipedia everytime we need the said article, we cache the tokens of the said entity/article in a file using the name of entity, so that we can directly fetch the token if it already has been processed. This helped us in saving alot of computation and api calls to wikipedia which is throttled.
 

In [ ]:
# User Configuration for pywikibot
family = 'wikipedia'
mylang = 'en'
usernames['wikipedia']['en'] = u'ExampleBot'

In [ ]:
# Wiki Parser Class
import pywikibot
import wiki2plain
import re
import random
from nltk.stem.porter import *
from nltk.corpus import stopwords

class WikiParser:
    def __init__(self):
        print("Making the Instance of Wiki parser.")
        self.site = pywikibot.Site('en', 'wikipedia')
        self.cache_stem = {}
        self.stemmer = PorterStemmer()
        self.stop_words = set(stopwords.words('english'))
        self.k = 50

    def getEntityTokens(self, wiki_entity):
        site = pywikibot.Site('en', 'wikipedia')
        page = pywikibot.Page(site, wiki_entity)  #here we just crawl for the new entry
        text = page.text
        wiki2plain_instance = wiki2plain.Wiki2Plain(text)  #make the text to plain text
        text = wiki2plain_instance.text
        text = text.lower()  #convert all the text to lower case. Case folding
        current_tokens = filter(None, re.split('\W+', text))  #get the tokens now
        current_tokens = [word for word in current_tokens if not word in self.stop_words]
        token_freq_map = {}
        for token in current_tokens:
            token = self.cacheInStem(token)
            if token not in token_freq_map:
                token_freq_map[token] = 1.0
            else:
                token_freq_map[token] += 1.0
        return token_freq_map

    def getCategoryForEntity(self, wiki_entity):
        site = pywikibot.Site('en', 'wikipedia')
        page = pywikibot.Page(site, wiki_entity)
        cat_values = page.categories()
        cat_list = list(cat_values)
        cat_names = []
        for cat in cat_list:
            if not cat.isHiddenCategory():
                cat_names.append(cat.title())
        return cat_names



    def getEntityforCategory(self, category):
        site = pywikibot.Site('en', 'wikipedia')
        catdata = pywikibot.Category(site, title=category)
        entities = catdata.articles()
        return self.getRefinedEntity(entities)

    def cacheInStem(self, token):
        if token not in self.cache_stem:
            self.cache_stem[token] = self.stemmer.stem(token)
        return self.cache_stem[token]

    def getRefinedEntity(self, entities):
        refinedEntity = []
        list_entities = list(entities)
        if len(list_entities) <= self.k:
            for entity in entities:
                refinedEntity.append(entity.title())
            return refinedEntity
        else:
            range_entity = range(0, len(list_entities))
            list_sample = random.sample(range_entity, self.k)
            for i in range(0, self.k):
                refinedEntity.append(list_entities[list_sample[i]].title())
            return refinedEntity


In [ ]:
# Wiki2PlainText

# from http://stackoverflow.com/questions/4460921/extract-the-first-paragraph-from-a-wikipedia-article-python

import re

class Wiki2Plain:
    def __init__(self, wiki):
        self.wiki = wiki

        self.text = wiki
        self.text = self.unhtml(self.text)
        self.text = self.unwiki(self.text)
        self.text = self.punctuate(self.text)

    def __str__(self):
        return self.text

    def unwiki(self, wiki):
        """
        Remove wiki markup from the text.
        """
        wiki = re.sub(r'(?i)\{\{IPA(\-[^\|\{\}]+)*?\|([^\|\{\}]+)(\|[^\{\}]+)*?\}\}', lambda m: m.group(2), wiki)
        wiki = re.sub(r'(?i)\{\{Lang(\-[^\|\{\}]+)*?\|([^\|\{\}]+)(\|[^\{\}]+)*?\}\}', lambda m: m.group(2), wiki)
        wiki = re.sub(r'\{\{[^\{\}]+\}\}', '', wiki)
        wiki = re.sub(r'(?m)\{\{[^\{\}]+\}\}', '', wiki)
        wiki = re.sub(r'(?m)\{\|[^\{\}]*?\|\}', '', wiki)
        wiki = re.sub(r'(?i)\[\[Category:[^\[\]]*?\]\]', '', wiki)
        wiki = re.sub(r'(?i)\[\[Image:[^\[\]]*?\]\]', '', wiki)
        wiki = re.sub(r'(?i)\[\[File:[^\[\]]*?\]\]', '', wiki)
        wiki = re.sub(r'\[\[[^\[\]]*?\|([^\[\]]*?)\]\]', lambda m: m.group(1), wiki)
        wiki = re.sub(r'\[\[([^\[\]]+?)\]\]', lambda m: m.group(1), wiki)
        wiki = re.sub(r'\[\[([^\[\]]+?)\]\]', '', wiki)
        wiki = re.sub(r'(?i)File:[^\[\]]*?', '', wiki)
        wiki = re.sub(r'\[[^\[\]]*? ([^\[\]]*?)\]', lambda m: m.group(1), wiki)
        wiki = re.sub(r"''+", '', wiki)
        wiki = re.sub(r'(?m)^\*$', '', wiki)

        return wiki

    def unhtml(self, html):
        """
        Remove HTML from the text.
        """
        html = re.sub(r'(?i)&nbsp;', ' ', html)
        html = re.sub(r'(?i)<br[ \\]*?>', '\n', html)
        html = re.sub(r'(?m)<!--.*?--\s*>', '', html)
        html = re.sub(r'(?i)<ref[^>]*>[^>]*<\/ ?ref>', '', html)
        html = re.sub(r'(?m)<.*?>', '', html)
        html = re.sub(r'(?i)&amp;', '&', html)

        return html

    def punctuate(self, text):
        """
        Convert every text part into well-formed one-space
        separate paragraph.
        """
        text = re.sub(r'\r\n|\n|\r', '\n', text)
        text = re.sub(r'\n\n+', '\n\n', text)

        parts = text.split('\n\n')
        partsParsed = []

        for part in parts:
            part = part.strip()

            if len(part) == 0:
                continue

            partsParsed.append(part)

        return '\n\n'.join(partsParsed)

    def image(self):
        """
        Retrieve the first image in the document.
        """
        # match = re.search(r'(?i)\|?\s*(image|img|image_flag)\s*=\s*(<!--.*-->)?\s*([^\\/:*?<>"|%]+\.[^\\/:*?<>"|%]{3,4})', self.wiki)
        match = re.search(r'(?i)([^\\/:*?<>"|% =]+)\.(gif|jpg|jpeg|png|bmp)', self.wiki)

        if match:
            return '%s.%s' % match.groups()

        return None


## Trivia Metric Calculator

In [ ]:
# Wiki-Trivia Metric Calculator
# Since by default the encoding scheme is based on the operating system, we will enforce the encoding scheme to be utf-8

import sys

reload(sys)
sys.setdefaultencoding('utf8')

import gensim
import heapq
import math
import Util as util
import pdb

class WikiTriviaMetricCalculator:
    def __init__(self):
        print "Inside the Initialization of the class: WikiTriviaExtractor"
        self.genism_model_filename = "GoogleNews-vectors-negative300.bin"
        self.model = gensim.models.KeyedVectors.load_word2vec_format(self.genism_model_filename, binary=True)
        self.global_idf = util.getglobalfreqdict("plainIdfIndex.txt")
        self.k_val = 10
        self.doc_size = 10000.0  #document size for the idf
        self.rare_term_freq = 10  #used for ignoring rarely occuring terms.
        print "Initialization Done"

    def GetModel(self):
        if self.model:
            return
        print 'Generating the Model'
        self.model = gensim.models.KeyedVectors.load_word2vec_format(self.genism_model_filename, binary=True)
        print 'Model Generated'

    # Get the top k tf idf tokens from the token freq map
    def getTopKTFIDFforEntity(self, token_frequency):
        entity_result = {}
        for token in token_frequency:
            if token not in self.model.vocab:
                continue
            tf = 1.0 + math.log10(token_frequency[token])
            global_freq = self.global_idf[token] if token in self.global_idf else 1.5
            if global_freq < self.rare_term_freq: #ignoring very rare terms
                continue
            entity_result[token] = tf * math.log10(self.doc_size/float(global_freq))
        return heapq.nlargest(self.k_val, entity_result, key=entity_result.get)



    def getEntitySimilarity(self,entity1, entity2):
        sim1 = self.getEntitySimilarityHelper(entity1, entity2)
        sim2 = self.getEntitySimilarityHelper(entity2, entity1)
        return ((sim1 + sim2) / 2.0)

    def getEntitySimilarityHelper(self, entity1, entity2):
        sim = 0.0
        if (len(entity1) < self.k_val or len(entity2) < self.k_val):
            return 0.0
        for i in range(0, self.k_val):
            current_max = self.model.similarity(entity1[i], entity2[0])
            for j in range(1, self.k_val):
                current_val = self.model.similarity(entity1[i], entity2[j])
                if current_val > current_max:
                    current_max = current_val
            sim += (self.k_val - i) * (current_max)
        sim = sim / ((self.k_val+1.0) * (float(self.k_val)))
        sim = sim * 2.0
        return sim

In [ ]:
# Algorithm Wrapper
import wiki_parser
import wiki_trivia_metric_calculator
import Util as util
import pdb
import os
import operator

#globals
category_entity_cache_dir = "catentcache/"
output_cache_dir = "outputCache/"
surprise_weight = 1.1

wiki_parser_instance = None
wiki_trivia_metric_calculator_instance = None


def triviaAlgorithm(search_entity, wiki_parser_instance, wiki_trivia_metric_calculator_instance):
    entity = util.searchWiki(search_entity)
    if not entity:
        return
    print "Entity Found: " + entity
    # Check for the Output Cache
    full_path = output_cache_dir + entity + ".txt"
    if not os.path.exists(output_cache_dir):
        os.makedirs(output_cache_dir)

    answer_mat = {}
    if os.path.isfile(full_path):
        open_output_file = open(full_path, "r")
        for line in open_output_file:
            trivia, score = line.split(":")
            answer_mat[trivia] = score
        open_output_file.close()
        answer_mat = sorted(answer_mat.items(), key=operator.itemgetter(1), reverse=True)
        return answer_mat


    # If the cache doesn't exist- Make the new one for the said entity
    entity_cats = wiki_parser_instance.getCategoryForEntity(entity)
    if not entity_cats:
        return
    if not os.path.exists(category_entity_cache_dir):
        os.makedirs(category_entity_cache_dir)
    for entity_cat in entity_cats:
        surprise_fact = surprise(entity, entity_cat, wiki_parser_instance, wiki_trivia_metric_calculator_instance)
        if surprise_fact:
            answer_mat[entity_cat.split(":")[1]] = surprise_fact
            cohes_score = cohesivness(entity_cat.split(":")[1], wiki_trivia_metric_calculator_instance)
            if cohes_score:
                answer_mat[entity_cat.split(":")[1]] *= cohes_score
            else:
                answer_mat[entity_cat.split(":")[1]] = 0.0
            print "<------------- ----------------->"
            print "Overall score for cat ", entity_cat, " is ", answer_mat[entity_cat.split(":")[1]]
            print "Ending     <------------- ----------------->"
    answer_mat = sorted(answer_mat.items(), key=operator.itemgetter(1), reverse=True)
    target = open(full_path, "w")
    for key in answer_mat:
        target.write(key[0] + ":" + repr(key[1]))
        target.write("\n")
    target.close()
    return answer_mat

def surprise(entity_input, entity_cat, wiki_parser_instance, wiki_trivia_metric_calculator_instance):
    sum = 0.0
    count = 0.0
    entity_input_tokens = wiki_parser_instance.getEntityTokens(entity_input)
    entity_input_top = wiki_trivia_metric_calculator_instance.getTopKTFIDFforEntity(entity_input_tokens)
    ent_cat_path = entity_cat.split(":")[1]
    path = category_entity_cache_dir + cleanifyPath(ent_cat_path) + "/"
    if os.path.exists(path):
        print "Reading from file "
        outer_list = []
        for(root, dirs, files) in os.walk(path):
            for file in files:
                if file.endswith('.txt'):
                    inner_list = []
                    current_file = open(os.path.join(root, file), "r")
                    for line in current_file:
                        line = line.replace('\n', '')
                        inner_list.append(line)
                    outer_list.append(inner_list)
        size_new = len(outer_list)
        for i in range(0, size_new):
            sum += wiki_trivia_metric_calculator_instance.getEntitySimilarity(entity_input_top, outer_list[i])
            count += 1.0
        answer = sum / count
        print "surprise for ", entity_cat, " is ", (1.0 / answer)
        return (1.0 / answer)

    new_entities = wiki_parser_instance.getEntityforCategory(entity_cat)
    if not new_entities:
        return

    for new_entity in new_entities:
        if new_entity != entity_input:
            new_entity_tokens = wiki_parser_instance.getEntityTokens(new_entity)
            new_entity_tokens_top = wiki_trivia_metric_calculator_instance.getTopKTFIDFforEntity(new_entity_tokens)
            new_entity_top_cache = category_entity_cache_dir + cleanifyPath(ent_cat_path) + "/"
            if not os.path.exists(new_entity_top_cache):
                os.makedirs(new_entity_top_cache)
            cache_file_name = new_entity_top_cache + cleanifyPath(new_entity) + ".txt"
            target = open(cache_file_name, "w")
            for top_token in new_entity_tokens_top:
                target.write(top_token)
                target.write("\n")
            target.close()
            sum += wiki_trivia_metric_calculator_instance.getEntitySimilarity(entity_input_top, new_entity_tokens_top)
            count += 1.0
    answer = sum / count
    print "surprise for " , entity_cat, " is ", (1.0/answer)
    return (1.0 / answer)

def cohesivness(entity_cat, wiki_trivia_metric_calculator_instance):
    sum = 0.0
    count = 0.0
    #answer = sum / count
    path = category_entity_cache_dir + cleanifyPath(entity_cat) + "/"
    outer_list = []
    for(root, dirs, files) in os.walk(path):
        for file in files:
            if file.endswith('.txt'):
                inner_list = []
                current_file = open(os.path.join(root, file), "r")
                for line in current_file:
                    line = line.replace('\n', '')
                    inner_list.append(line)
                outer_list.append(inner_list)
    size_new = len(outer_list)
    for i in range(0, size_new):
        for j in range(i+1, size_new):
                sum += wiki_trivia_metric_calculator_instance.getEntitySimilarity(outer_list[i], outer_list[j])
                count += 1.0
    if count == 0.0:
        return 0.0
    answer = sum / count
    print "Cohes is for cat ", entity_cat, " is ", answer
    return answer

def cleanifyPath(path):
    path = ''.join(e for e in path if e.isalnum())
    return path


## Main calling Function Entry Points

In [1]:
# Main Algorithm Calling Function
import algorithm_wrapper
import wikipedia as wiki
import pdb
import wiki_parser
import wiki_trivia_metric_calculator


if __name__ == "__main__":
    wiki_parser_instance = wiki_parser.WikiParser()
    wiki_trivia_metric_calculator_instance = wiki_trivia_metric_calculator.WikiTriviaMetricCalculator()
    print "Init done"
    target = open("input.txt", "r")
    for line in target:
        line = line.replace('\n', '')
        print algorithm_wrapper.triviaAlgorithm(line, wiki_parser_instance, wiki_trivia_metric_calculator_instance)
    target.close()

messi


TypeError: triviaAlgorithm() takes exactly 3 arguments (1 given)

# Web Interface

## Evaluation 

TBD

## Conclusion and Future Work

Majority of the search engines enlist the results which most of the people know or plethora of the webpages reflect thus always passing the common and popular results. Moreover, the search engines results try to fulfil the information need of the user. But it might be fun and learning experience to suplement this information with trivia facts, which might surprise and entertain you. In the work presented above we provided a metric to calculate the trivia-worthiness of the facts obtained from the wikipedia. Since the evaluation part of the work couldn't be performed given the timelines and the fact that a trivia can be subjective- meaning depending in the person. This work has many benefits,as Tsurel puts it "while seemingly lighthearted, can lead
to higher engagement of users searching for named entities.
If successful, even a small impact on this type of queries can
translate into a substantial improvement in user experience,
and possibly transfer to other domains of human activity,
like education.".

Furthermore, to engage the user in the trivia facts and to see how our algorithm works, we developed a game based on the trivia obtained using our algorithm. We take the top five trivia facts for each entity our system has seen till now and then develop five True or False questions. Since we know the ground truth about the entity and its corresponding trivia. We form the question based on the ground truth, we involve the False answer based question by shuffling the entity and corresponding category so that now an entity can either exist in the category or not. As we know the correct answer whether that person or entity esxists in the category we can easily evaluate the answer and give a score to the user. This helps to  enhanc the trivia knowledge related to the random person or entity.

Future work can be in two directions, one taking the data from some other source. As we know that the wikipedia is crowd sourced website and the chances of a category being a fact about an entity/person is debatable. Other direction could be coming up an new metric to calculate the trivia-worthiness of the fact.


## References
[1] Mika, Peter. "Entity search on the web." Proceedings of the 22nd International Conference on
World Wide Web. ACM, 2013.

[2] Yin, Xiaoxin, and Sarthak Shah. "Building taxonomy of web search intents for name entity queries."
Proceedings of the 19th international conference on World wide web. ACM, 2010.

[3] Sergey Chernov, Tereza Iofciu, Wolfgang Nejdl, and Xuan Zhou. Extracting semantics relationships
between Wikipedia categories. SemWiki, 206, 2006.

[4] Abhay Prakash, Manoj K. Chinnakotla, Dhaval Patel, and Puneet Garg. Did you know?: Mining
interesting trivia for entities from wikipedia. In Proceedings of the 24th International Conference
on Artificial Intelligence, IJCAI'15, pages 3164--3170. AAAI Press, 2015.

[5] Matthew Merzbacher. Automatic generation of trivia questions. In International Symposium on
Methodologies for Intelligent Systems, pages 123-130.Springer, 2002.

[6] Tsurel, David, et al. "Fun Facts: Automatic Trivia Fact Extraction from Wikipedia." WSDM 2017.

[7] Wiki2Plain Reference

[8] https://code.tutsplus.com/series/creating-a-web-app-from-scratch-using-python-flask-and-mysql--cms-827

[9] https://www.w3schools.com/html/html_responsive.asp

[10] http://flask.pocoo.org/

[11] http://tutorialzine.com/2015/02/freebie-7-responsive-header-templates/ (Header Template)

[12] http://www.flashbynight.com/tutes/mcqquiz/ (Quiz Template)

[13] http://fontawesome.io/

[14] http://nbviewer.jupyter.org/ (Notebook Viewer)

[15] https://github.com/singhsume123/TSSE (Carousel)